In [38]:
import numpy as np
import pandas as pd

<img src='Images/ml_2.png'>

Для начала составим обычную матрицу наблюдений A, расположив векторы в столбцах. Обратите внимание, что вектор из 1 мы не будем добавлять в матрицу (за нас это сделает генератор полиномиальных признаков):

In [39]:
A = np.array([
    [1,3,-2,1,5,13,1],
    [3,4,5,-2,4,11,3],
    [4,5,2,2,6,8,-1]
]).T

A

array([[ 1,  3,  4],
       [ 3,  4,  5],
       [-2,  5,  2],
       [ 1, -2,  2],
       [ 5,  4,  6],
       [13, 11,  8],
       [ 1,  3, -1]])

Затем импортируем класс PolynomialFeatures из библиотеки sklearn. Создадим объект этого класса, указав при инициализации степень полинома равной 2. Также укажем, что нам нужна генерация столбца из 1 (параметр include_bias=True):

In [40]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=True)

Осталось только вызвать метод fit_transform() от имени этого объекта и передать в него нашу матрицу наблюдений A. Для удобства выведем результат в виде DataFrame:

In [41]:
A_poly = poly.fit_transform(A)
display(pd.DataFrame(A_poly))

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,3.0,4.0,1.0,3.0,4.0,9.0,12.0,16.0
1,1.0,3.0,4.0,5.0,9.0,12.0,15.0,16.0,20.0,25.0
2,1.0,-2.0,5.0,2.0,4.0,-10.0,-4.0,25.0,10.0,4.0
3,1.0,1.0,-2.0,2.0,1.0,-2.0,2.0,4.0,-4.0,4.0
4,1.0,5.0,4.0,6.0,25.0,20.0,30.0,16.0,24.0,36.0
5,1.0,13.0,11.0,8.0,169.0,143.0,104.0,121.0,88.0,64.0
6,1.0,1.0,3.0,-1.0,1.0,3.0,-1.0,9.0,-3.0,1.0


### Матрица $A_{\text{poly}}$, описание столбцов:

- **Столбец 0** — единичный, он отвечает за слагаемое с нулевой степенью полинома (любое число в степени $0$ даёт единицу).
- **Столбцы 1, 2 и 3** — это наши исходные признаки (векторы $x_1, x_2$ и $x_3$).
- **Столбцы 4, 5 и 6** — произведения первого столбца со всеми столбцами:  
  $x_1 \cdot x_1$, $x_1 \cdot x_2$, $x_1 \cdot x_3$ соответственно.
- **Столбцы 7 и 8** — произведения второго столбца со столбцами 2 и 3:  
  $x_2 \cdot x_2$, $x_2 \cdot x_3$.
- **Столбец 9** — произведение третьего столбца с самим собой:  
  $x_3 \cdot x_3 = x_3^2$.

---

Таким образом, при генерации полиномиальных признаков объект `PolynomialFeatures` сначала создаёт исходные факторы, затем умножает каждый из них на все факторы и повторяет процедуру. При этом, если комбинация $x_i \cdot x_j$ уже была сгенерирована ранее, то комбинация $x_j \cdot x_i$ не рассматривается.


А теперь построим модель полиномиальной регрессии на реальных данных.

Возьмём все те же данные о стоимости жилья в районах Бостона. Будем использовать следующие четыре признака: LSTAT, CRIM, PTRATIO и RM. С их помощью мы построим полиномиальную регрессию от первой до пятой степени включительно, а затем сравним результаты по значению средней абсолютной процентной ошибки (MAPE).

Чтобы не дублировать код, объявим функцию polynomial_regression(). Она будет принимать на вход матрицу наблюдений, вектор ответов и степень полинома, а возвращать матрицу с полиномиальными признаками, вектор предсказаний и коэффициенты регрессии, найденные по МНК:

In [42]:
from Math_algorithms import polynomial_regression

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
# 'header=None' указывает, что в файле нет строки заголовков, и мы задаем имена столбцов вручную
# 'delimiter=r"\s+"' указывает на то, что разделителем в файле являются пробелы (или любые пробелы)
# 'names=column_names' задаёт имена столбцов из списка 'column_names'
boston_data = pd.read_csv('Data/housing.csv', header=None, delimiter=r"\s+", names=column_names)
display(boston_data.head(5))

# Делим выборку
A = boston_data[['LSTAT', 'PTRATIO', 'RM', 'CRIM']]
y = boston_data[['PRICE']]

A_poly1, y_pred1, w_hat1 = polynomial_regression(A, y, 1)
A_poly2, y_pred2, w_hat2 = polynomial_regression(A, y, 2)
A_poly3, y_pred3, w_hat3 = polynomial_regression(A, y, 3)
A_poly4, y_pred4, w_hat4 = polynomial_regression(A, y, 4)
A_poly5, y_pred5, w_hat5 = polynomial_regression(A, y, 5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


Посмотрим на качество построенных регрессий, вычислив метрику:

In [43]:
from sklearn.metrics import mean_absolute_percentage_error
 
print('MAPE для полинома 1-й степени {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred1)*100))
print('MAPE для полинома 2-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred2)*100))
print('MAPE для полинома 3-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred3)*100))
print('MAPE для полинома 4-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred4)*100))
print('MAPE для полинома 5-й степени  {:.2f}%'.format(mean_absolute_percentage_error(y, y_pred5)*100))

MAPE для полинома 1-й степени 18.20%
MAPE для полинома 2-й степени  13.41%
MAPE для полинома 3-й степени  12.93%
MAPE для полинома 4-й степени  10.74%
MAPE для полинома 5-й степени  425.50%


Полиномиальная регрессия первой степени (линейная регрессия) показывает наименьшее качество предсказания, так как зависимость между факторами и целевым признаком нелинейная. С повышением степени полинома процентная ошибка на обучающей выборке вроде бы падает, однако для полинома пятой степени она резко возрастает. Это означает, что модель вообще не описывает зависимость в исходных данных — её прогноз не имеет никакого отношения к действительности.

Интерес представляет полином 5 степени, разобраться почему процент такой резко высокий, поможет объяснение ниже:

In [44]:
display(pd.DataFrame(w_hat5).describe())

,PRICE
count,126.000000
mean,409.591413
std,12248.037174
min,-70520.884813
25%,-1.028490
50%,0.000006
75%,0.434758
max,116177.366878


Видно, что в степенях минимального и максимального коэффициентов явно что-то не так — коэффициенты слишком огромные (исчисляются миллионами).

Теперь давайте взглянем на корреляционную матрицу для факторов, на которых мы строим полином пятой степени. Корреляцию со столбцом из единиц считать бессмысленно, поэтому мы не будем его рассматривать. Для удобства расчёта матрицы корреляций обернём матрицу  в DataFrame и воспользуемся методом corr():

In [45]:
# считаем матрицу корреляций (без столбца из единиц)
C = pd.DataFrame(A_poly5[:, 1:]).corr()

# считаем ранг корреляционной матрицы
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(C))

# считаем количество факторов (не включая столбец из единиц)
print('Количество факторов:', A_poly5[:, 1:].shape[1])

Ранг корреляционной матрицы: 110
Количество факторов: 125


Мы нашли корень проблемы: ранг корреляционной матрицы — 110, в то время как общее количество факторов (не считая единичного столбца) — 125, то есть ранг корреляционной матрицы не максимален. Это значит, что в корреляционной матрице присутствуют единичные корреляции, а в исходной матрице — линейно зависимые столбцы.

Кстати, заметим, что, например, для полинома четвёртой степени ранг матрицы корреляций максимален, то есть равен количеству факторов (не включая единичный столбец):

In [46]:
# считаем матрицу корреляций (без столбца из единиц)
C = pd.DataFrame(A_poly4[:, 1:]).corr()
# считаем ранг корреляционной матрицы
print('Ранг корреляционной матрицы:', np.linalg.matrix_rank(C))
# считаем количество факторов (не включая столбец из единиц)
print('Количество факторов:', A_poly4[:, 1:].shape[1])

Ранг корреляционной матрицы: 69
Количество факторов: 69


Поэтому и коэффициенты регрессии полинома четвёртой степени находятся в адекватных пределах.

In [47]:
display(pd.DataFrame(w_hat4).describe())

,PRICE
count,70.000000
mean,-50.809065
std,886.564118
min,-6918.586311
25%,-0.187936
50%,-0.000796
75%,0.322202
max,2304.941937


А теперь посмотрим, что будет, если использовать для построения полиномиальной регрессии реализацию из библиотеки sklearn.

In [48]:
from Math_algorithms import polynomial_regression_sk

A = boston_data[['LSTAT', 'PTRATIO', 'RM', 'CRIM']]
y = boston_data[['PRICE']]

for k in range(1, 6):
    A_poly, y_pred, w_hat = polynomial_regression_sk(A, y, k)
    print(
        "MAPE для полинома степени {} — {:.2f}%, СКО — {:.0f}".format(
            k, mean_absolute_percentage_error(y, y_pred)*100, w_hat.std()
        )
    )

MAPE для полинома степени 1 — 18.20%, СКО — 2
MAPE для полинома степени 2 — 13.41%, СКО — 5
MAPE для полинома степени 3 — 12.93%, СКО — 9
MAPE для полинома степени 4 — 10.74%, СКО — 304
MAPE для полинома степени 5 — 8.98%, СКО — 290


«магия» sklearn — построение полинома пятой степени прошло успешно.

Секрет в том, что в sklearn для построения линейной регрессии используется не сама матрица наблюдений A, а её сингулярное разложение, которое гарантированно является невырожденным — из него исключаются линейно зависимые факторы, но такие данные могут быть сомнительными.

Примечание: Чем выше степень полинома, тем выше шанс переобучения!

### Итог: 

- Модель полиномиальной регрессии — более общий случай линейной регрессии, в котором зависимость целевой переменной от факторов нелинейная.

- Поиск коэффициентов полинома аналогичен линейной регрессии — решение неоднородной СЛАУ.

- Возможна ситуация, когда какие-то сгенерированные полиномиальные факторы могут линейно выражаться через другие факторы. Тогда ранг корреляционной матрицы будет меньше числа факторов и поиск по классическому МНК-алгоритму не будет успешным.

- В sklearn для решения последней проблемы предусмотрена защита — использование сингулярного разложения матрицы A. Однако данная защита не решает проблемы неустойчивости коэффициентов регрессии.

- Полиномиальная регрессия имеет сильную склонность к переобучению: чем выше степень полинома, тем сложнее модель и выше риск переобучения.

# Практика

Построена модель полиномиальной регрессии следующего вида:

$$
y = 10.4 + 8 \cdot x_1 + 0.5 \cdot x_2 + 3 \cdot x_1^2 + 0.4 \cdot x_2^2 + 0 \cdot x_1 x_2
$$

Поступило новое наблюдение, которое характеризуется вектором:

$$
x_{\text{new}} = \begin{pmatrix} x_{1\text{new}} \\ x_{2\text{new}} \end{pmatrix} = \begin{pmatrix} 1 \\ 4 \end{pmatrix}
$$

Сделайте прогноз целевой переменной с помощью полученной полиномиальной регрессии.  
Ответ **округлите до первого знака после точки-разделителя**.


In [49]:
x_new = np.array([1,4])
# x_new[0] = 1 (x_1)
# x_new[1] = 4 (x_2)
y = (10.4 + 8 * x_new[0] + 0.5 * x_new[1] + 3 * x_new[0]**2 + 0.4 * x_new[1]**2 + 0 * x_new[0] * x_new[1])
print(y.round(1))

29.8


С помощью классического МНК найдите коэффициенты полиномиальной регрессии, если используется полином второй степени и задан фактор $\vec{x}$ и целевая переменная $\vec{y}$:

$$
\vec{y} = w_0 + w_1 \vec{x} + w_2 \vec{x}^2
$$

Где:

$$
\vec{x} =
\begin{pmatrix}
1 \\
3 \\
-2 \\
9
\end{pmatrix}, \quad
\vec{y} =
\begin{pmatrix}
3 \\
7 \\
-5 \\
21
\end{pmatrix}
$$

В качестве ответа приведите координаты вектора коэффициентов $w_0$, $w_1$, $w_2$, **округлив их до первого знака после точки-разделителя**.


In [50]:
A = np.array([
    [1,1,1,1],
    [1,3,-2,9],
    [1**2,3**2,-2**2,9**2]
    ]).T

print(A)

y = np.array([3,7,-5,21])

w_hat = np.linalg.inv(A.T @ A) @ A.T @ y
w_0 = w_hat[0]
w_1 = w_hat[1]
w_2 = w_hat[2]
display(f'w_0:{w_0:.1f}, w_1:{w_1:.1f}, w_2:{w_2:.1f}')

[[ 1  1  1]
 [ 1  3  9]
 [ 1 -2 -4]
 [ 1  9 81]]


'w_0:0.1, w_1:2.5, w_2:-0.0'

# Регуляризация

Обучим модель полиномиальной регрессии третьей степени. Будем использовать данные о жилье в Бостоне и возьмём следующие четыре признака: LSTAT, CRIM, PTRATIO и RM.

Для оценки качества модели будем использовать кросс-валидацию и сравнивать среднее значение метрики на тренировочных и валидационных фолдах. Кросс-валидацию организуем с помощью функции cross_validate из модуля model_selection:

In [51]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

В качестве метрики используем среднюю абсолютную процентную ошибку — MAPE.

In [52]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]

# Добавляем полиноминальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# Создаем модель линейной регрессии
lr = LinearRegression()

# Оцениваем модель на кросс-валидации, метрика - МАРЕ
cv_results = cross_validate(
    lr, # Модель линейной регрессии
    X, # Факторы
    y, # Целевая переменная
    scoring='neg_mean_absolute_percentage_error', # метрика
    cv=5, # 5 фолдов
    return_train_score=True # Возвращаем результаты валидации
    )

print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	

MAPE на тренировочных фолдах: 12.64 %
MAPE на валидационных фолдах: 24.16 %


Что мы видим? Даже при, казалось бы, небольшой, третьей степени полинома мы получили переобучение: на тренировочной выборке MAPE=12.64%, а вот на тестовой — MAPE=24.16%. Показатели качества отличаются практически в два раза, что говорит о высоком разбросе модели.

Для того, чтобы справиться с этой ситуацией мы и воспользуемся методом регуляризации.

Регуляризация — это способ уменьшения переобучения моделей машинного обучения путём намеренного увеличения смещения модели для уменьшения её разброса.

Регуляризация для линейной регрессии преследует сразу несколько целей. Однако далее мы увидим, что все эти цели на самом деле взаимосвязаны:

- предотвратить переобучение модели;
- включить в функцию потерь штраф за переобучение;
- обеспечить существование обратной матрицы $(A^T A)^-1$;
- не допустить огромных коэффициентов модели.

---

Большие значения весов — прямое свидетельство переобучения модели линейной регрессии и её нестабильности. Идея регуляризации состоит в наложении ограничения на вектор весов (часто говорят — наложение штрафа за высокие веса). В качестве штрафа принято использовать норму вектора весов.

---

# L2 - RIDGE

Очевидно, что матрица $A^T A$ вырождена: её второй и третий столбцы являются пропорциональными с коэффициентом 2. Значит, наша классическая формула МНК  (без сингулярного разложения) не сработает.

In [53]:
# матрица наблюдений (включая столбец единиц)
# A = np.array([
#     [1, 1, 1, 1, 1],
#     [1, 0, -3, 2, 4],
#     [2, 0, -6, 4, 8]
# ]).T

# # вектор целевого признака
# y = np.array([4, 3, -4, 2, 7])
# # получаем оценку коэффициентов регрессии по МНК
# w_hat = np.linalg.inv(A.T @ A) @ A.T @ y
# print(w_hat)

Мы ожидаемо получили ошибку, говорящую о том, что матрица A вырождена. 

Попробуем найти вектор оценок весов w ridge по формуле:

In [54]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T

# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])

# единичная матрица
E = np.eye(3)

# коэффициент регуляризации
alpha = 5

# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
w_hat_ridge = np.linalg.inv(A.T @ A + alpha * E) @ A.T @ y
print(w_hat_ridge)

[0.6122449  0.29387755 0.5877551 ]


За реализацию линейной регрессии с $L_2$-регуляризацией в sklearn отвечает класс Ridge. Основной параметр модели, на который стоит обратить внимание — alpha, коэффициент регуляризации из формулы Тихонова.

In [55]:
from sklearn.linear_model import Ridge

Давайте обучим модель для решения нашей последней задачи, а затем проверим коэффициенты регрессии. Так как мы заранее заложили в матрицу A столбец из единиц, то, чтобы получить корректное решение, параметр fit_intercept следует установить в значение False.

In [56]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T

# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])

# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
ridge = Ridge(alpha=5, fit_intercept=False)
ridge.fit(A, y)

# Результаты коэффициентов
print(ridge.coef_)

[0.6122449  0.29387755 0.5877551 ]


Получили тот же самый результат, что и раньше.

Наконец, посмотрим, как регуляризация поможет побороть переобучение модели полиномиальной регрессии на наборе данных о домах в Бостоне. Используем те же самые признаки: LSTAT, CRIM, PTRATIO и RM.

Сразу cтоит отметить, что для успешной сходимости численных методов оптимизации, которые используются для решения задачи условной оптимизации, необходима стандартизация (нормализация) исходных данных, которая не требовалась для аналитического МНК в классической линейной регрессии (LinearRegression).

Примечание: Здесь под стандартизацией понимается именно приведение распределения признака к нулевому среднему и единичному стандартному отклонению (StandardScaler).

In [57]:
from sklearn.preprocessing import StandardScaler

Воспользуемся моделью полиномиальной регрессии третьей степени с регуляризацией Тихонова (коэффициент регуляризации возьмём равным 20) и проверим её качество на кросс-валидации по метрике MAPE.

In [58]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]

# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()

# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L2-регуляризацией
ridge = Ridge(alpha=20, solver='svd')

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(ridge, 
                            X, 
                            y, 
                            scoring='neg_mean_absolute_percentage_error', 
                            cv=5, 
                            return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 12.54 %
MAPE на валидационных фолдах: 17.02 %


Нам удалось уменьшить ошибку (MAPE) на валидационных фолдах кросс-валидации с 24.16% до 17.02% и сократить разницу в метриках, тем самым уменьшив разброс ответов модели.

# Практика

Вычислите коэффициенты линейной регрессии с \( L_2 \)-регуляризацией, используя аналитическую формулу Тихонова, если:

$$
\vec{x}_1 =
\begin{pmatrix}
5 \\
9 \\
4 \\
3 \\
5
\end{pmatrix}, \quad
\vec{x}_2 =
\begin{pmatrix}
15 \\
18 \\
18 \\
19 \\
19
\end{pmatrix}, \quad
\vec{x}_3 =
\begin{pmatrix}
7 \\
6 \\
7 \\
7 \\
7
\end{pmatrix}, \quad
\vec{y} =
\begin{pmatrix}
24 \\
22 \\
35 \\
33 \\
36
\end{pmatrix}
$$

Коэффициент регуляризации (alpha = 1).

---

В качестве ответа приведите значения полученных коэффициентов линейной регрессии, **округлив их до второго знака после точки-разделителя**.


In [59]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1,1,1,1,1],
    [5,9,4,3,5],
    [15,18,18,19,19],
    [7,6,7,7,7]
]).T

# вектор целевого признака
y = np.array([24,22,35,33,36])

# единичная матрица
E = np.eye(4)

# коэффициент регуляризации
alpha = 1

# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
w_hat_ridge = np.linalg.inv(A.T @ A + alpha * E) @ A.T @ y
print(w_hat_ridge[0].round(2))
print(w_hat_ridge[1].round(2))
print(w_hat_ridge[2].round(2))
print(w_hat_ridge[3].round(2))

-0.09
-1.71
1.91
0.73


# L1 - LASSO

В sklearn $L_1$-регуляризация реализована в классе Lasso, а оптимизационная задача решается алгоритмом координатного спуска (Coordinate Descent).

In [60]:
from sklearn.linear_model import Lasso

Давайте посмотрим, как работает Lasso на примере:

Попробуем найти коэффициенты регрессии с помощью $L_1$-регуляризации. Для этого подадим нашу матрицу наблюдений A и вектор целевого признака $\hat{y}$ в модель Lasso.

In [61]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4,3,-4,2,7])

# получаем оценку коэффициентов регрессии с помощью L1-регуляризации
lasso = Lasso(alpha=0.1, fit_intercept=False)
lasso.fit(A, y)

# Получаем коэффициенты
print(lasso.coef_)

[1.14925373 0.         0.71921642]


Теперь давайте применим $L_1$-регуляризацию к нашей полиномиальной модели третьей степени, прогнозирующей типичную цену на дома в районах Бостона.

Так как метод координатного спуска, который применяется для поиска коэффициентов, является численным, то необходима стандартизация исходных данных, чтобы обеспечить ему сходимость. Возьмём в качестве коэффициента регуляризации $alpha = 0.1$ и проверим качество полученной модели с помощью кросс-валидации по метрике MAPE:

In [63]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]

# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()

# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L1-регуляризацией
lasso = Lasso(alpha=0.1, max_iter=10_000)

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(
    lasso,
    X,
    y,
    scoring='neg_mean_absolute_percentage_error',
    cv=5,
    return_train_score=True
)

print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 12.44 %
MAPE на валидационных фолдах: 16.44 %


Видим, что с помощью $L_1$-регуляризации удалось уменьшить ошибку модели (MAPE) на валидационных фолдах с 24.16% до 16.44% и сократить разницу в метриках на тренировочных и валидационных фолдах даже лучше, чем с этим справилась $L_2$-регуляризация. Однако на самом деле мы просто удачно выбрали коэффициент регуляризации — при других значениях могли получиться совершенно другие результаты.

# ELASTIC-NET (Комбинация регуляризации L-1 и L-2)

В sklearn эластичная сетка реализована в классе ElasticNet из пакета с линейными моделями — linear_model. За коэффициент $\alpha$ отвечает параметр alpha, за коэффициент $\lambda$ — L1_ratio.

Некоторые рекомендации от разработчиков ElasticNet:

1. Использование параметра l1_ratio < 0.01 приводит к нестабильным результатам!
2. Вместо использования ElasticNet с alpha=0 лучше используйте LinearRegression, так как там применяется аналитическое решение, которое позволяет получать более точные решения, чем численный координатный спуск.

In [64]:
from sklearn.linear_model import ElasticNet

Рассмотрим пример работы с Elastic-Net, а затем применим эту модель к нашей задаче о домах в Бостоне.

Решим задачу с тремя комбинациями коэффициентов регуляризации:

1. $\alpha$ = 0.1, $\lambda$ = 0.2
2. $\alpha$ = 0.1, $\lambda$ = 0.7
3. $\alpha$ = 0.1, $\lambda$ = 1

### Случай 1.

In [65]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T

# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])

# получаем оценку коэффициентов регрессии
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.2, fit_intercept=False)
elastic_net.fit(A, y)

# Получаем коэффициенты
print(elastic_net.coef_)

[1.13492457 0.19525842 0.6237965 ]


Обратим внимание, что зануления коэффициентов коллинеарных факторов $x_1$ и $x_2$ не произошло. Каждый из них вошёл в уравнение регрессии с ненулевым коэффициентом.

### Случай 2.

In [66]:
# получаем оценку коэффициентов регрессии
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.7, fit_intercept=False)
elastic_net.fit(A, y)

# Получаем коэффициенты
print(elastic_net.coef_)

[1.14379753 0.         0.71993025]


Обратим внимание, что произошло зануление коэффициентов. Это неспроста, так как мы понизили влияние $L_2$-регуляризации и одновременно повысили влияние $L_1$-регуляризации, которая, как мы уже знаем, приводит к исключению линейно зависимых факторов.

### Случай 3.

In [67]:
# получаем оценку коэффициентов регрессии
elastic_net = ElasticNet(alpha=0.1, l1_ratio=1, fit_intercept=False)
elastic_net.fit(A, y)

# Получаем коэффициенты
print(elastic_net.coef_)

[1.14925373 0.         0.71921642]


В округлениях значения не заметно, однако если присмотреться к коэффициентам более внимательно, можно увидеть, что мы получили в точности те же значения, которые получали для модели Lasso в примере № 2. Неудивительно, ведь мы обнулили влияние $L_2$-регуляризации, выставив l1_ratio=1. По сути, мы использовали чистую модель Lasso.

Возникает вопрос: какой набор коэффициентов линейной регрессии всё-таки подходит лучше?

Ответить на него можно, только вычислив метрику качества и сравнив ошибки прогнозов каждой из полученных моделей!

Осталось только попробовать применить Elastic-Net к данным о недвижимости в Бостоне.

Как и для других моделей с регуляризацией, для Elastic-Net также лучше заранее позаботиться о стандартизации данных. В качестве коэффициентов регуляризации возьмём $\alpha$=0.1, $\lambda$=0.5. Качество модели проверим с помощью кросс-валидации на пяти фолдах, метрика — MAPE.

In [75]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]

# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()

# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L1- и L2-регуляризациями
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(
    elastic_net, 
    X, 
    y, 
    scoring='neg_mean_absolute_percentage_error', 
    cv=5, 
    return_train_score=True
    )

print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100)) 

MAPE на тренировочных фолдах: 12.65 %
MAPE на валидационных фолдах: 15.70 %


Итак, Elastic-Net позволил нам уменьшить значение MAPE на валидационных фолдах с 24.16% до 15.7%. Отличный результат! Он получился лучше, чем у моделей Ridge и Lasso, но опять же, так бывает не всегда.

На практике при использовании моделей с регуляризацией стоит подбирать значения коэффициентов регуляризации с помощью методов подбора гиперпараметров. Только после подбора гиперпараметров можно сделать вывод, какая из моделей показывает наилучшие результаты для решения конкретной задачи.